## Udacity Project 1: Political Instability Prediction in Africa and the Middle East with a Random Forest Model

#### Data and Package Loading

In [0]:
# Install the necessary packages
%pip install shap

In [0]:
# Install NumPy version 1.24.0 for compatibility with the SHAP-package
%pip install numpy==1.24.0

In [0]:
# Proceed only when the NumPy version given below is indeed 1.24.0
import numpy
print(numpy.__version__)

In [0]:
# Import the necessary packages and functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, classification_report, f1_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [0]:
# Read in the data from a csv downloaded from the World Bank database
df = pd.read_csv(
    '/Workspace/Users/j60849@eon.com/Udacity_Project_1/Dataset_World_Bank.csv',
    sep='","',
    skipinitialspace=True,
    engine='python',
    quotechar='"'
)

# Print the data to evaluate pre-processing steps
df.head()

#### Data pre-processing

In [0]:
# Strip the column headers of their double quotes
df.columns = df.columns.str.strip('"')

In [0]:
# Remove double quotes from the data
def quote_remover(df):
    "This function removes double quotes from all entries of a pandas dataframe"
    for col in df.columns:
        df[col] = df[col].str.replace('"', '')
    return df

df = quote_remover(df)

In [0]:
# Single out the country name from the country code in the first column
df['Country_Name'] = df.iloc[:, 0].str.split(',').str[0]

In [0]:
# Select a subset of the data with which the modelling can take place
df_model = df[['Country_Name', 'Series Name', 'Time', 'Value']]

In [0]:
# Pivot the data so that each unique Series Name becomes a separate column
df_model = df_model.drop_duplicates(subset=['Country_Name', 'Time', 'Series Name'])
df_pivot = df_model.pivot(index=['Country_Name', 'Time'], columns='Series Name', values='Value').reset_index()

In [0]:
# Change blank spaces into null values
df_pivot = df_pivot.replace(r'^\s*$', None, regex=True)

In [0]:
# Inspect the dataframe with .info() to see the number of missing values per column
df_pivot.info()

In [0]:
# Change datatype of all numerical columns to float
for column in df_pivot.columns:
    if df_pivot[column].dtype == 'object' and column != 'Country_Name' and column != 'Time':
        df_pivot[column] = df_pivot[column].astype('float')
df_pivot.info()

In [0]:
# Drop the columns with too many few data points reported
df_pivot.drop(['Arms exports (SIPRI trend indicator values)', 'Arms imports (SIPRI trend indicator values)', 'Central government debt, total (% of GDP)', 'International migrant stock (% of population)', 'School enrollment, secondary, male (% net)'], inplace=True, axis=1)

In [0]:
# Drop the rows for 2024, since too much data from last year has not been processed well into the World Bank database yet
df_pivot = df_pivot[df_pivot['Time'] != '2024']

In [0]:
# Count number of null values per country to see whether some countries have not enough data to be eligible for the model
null_counts = df_pivot.groupby('Country_Name').apply(lambda x: x.isnull().sum().sum())
null_counts_df = null_counts.to_frame(name='null_count').reset_index()
display(null_counts_df)

In [0]:
# Remove some small (island) nations from the dataframe that lack too much data
df_pivot = df_pivot[
    (df_pivot['Country_Name'] != 'Comoros') &
    (df_pivot['Country_Name'] != 'Eritrea') &
    (df_pivot['Country_Name'] != 'Sao Tome and Principe') &
    (df_pivot['Country_Name'] != 'Seychelles') &
    (df_pivot['Country_Name'] != 'Somalia') &
    (df_pivot['Country_Name'] != 'South Sudan') &
    (df_pivot['Country_Name'] != 'Syrian Arab Republic') &
    (df_pivot['Country_Name'] != 'West Bank and Gaza')
]

In [0]:
# Construct conflict indicator out of Political Stability estimate (conflict when the estimate is below -1, since then, the perception of political stability and absence of violence is at least one standard deviation away from the global mean)
df_pivot['IND_CONFLICT'] = np.where((df_pivot['Political Stability and Absence of Violence/Terrorism: Estimate'] < -1), 1, 0)
display(df_pivot)

#### Data Exploration

In [0]:
# Generate some summary statistics of the data
df_pivot.describe()

In [0]:
# Make histograms of all important variables to see their distributions
df_pivot.hist(bins=50, figsize=(25,15))
plt.show()

As becomes clear from the distribution plot above, access to electricity is skewed towards 100, foreign direct investment contains some outliers and all other variables except population growth, GDP growth and government effectiveness are right skewed

In [0]:
# Calculate the correlation matrix to signal potential multicollinearity
plt.figure(figsize=(25, 15))
correlation_matrix = df_pivot.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [0]:
# Control of Corruption and Government Effectiveness are multicollinear with a pairwise correlation of 0.9. Therefore, remove these variables from the data
df_pivot.drop(['Government Effectiveness: Estimate', 'Control of Corruption: Estimate'], inplace=True, axis=1)

In [0]:
# Check multicollinearity also with the VIF to identify non-linear and non-pairwise collinearity

# Multicollinearity Test: VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import pandas as pd

# Ensure all columns are numeric and handle missing values
X = df_pivot.drop(['IND_CONFLICT', 'Country_Name', 'Time'], axis=1).apply(pd.to_numeric, errors='coerce').fillna(0)

# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["Feature"] = X.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))] 

print(vif_data)

All VIF-scores are lower than 10, so there is no problematic multicollinearity present.

In [0]:
# Show the correlation of the features with the target to identify potentially strong features and weak features
correlation_matrix = df_pivot.corr()
correlation_matrix["IND_CONFLICT"].sort_values(ascending=False)

As is shown in the correlation matrix above, the individual variables seem to not have a strong correlation with the target variable IND_CONFLICT, except maybe merchandise trade and inflation. The correlation of political stability is understandingly strong, since the IND_CONFLICT target variable is based on this variable. Therefore, the Political Stability variable will not be included in the final model

#### Modelling

While a logistic regression could be used, since the target variable IND_CONFLICT is binary, opt for a random forest because of class imbalance (the number of conflicts is relatively small compared to the number of country-year pairs) and better handling of features with skewed distributions.

In [0]:
# Define the feature set and the target variable
X = df_pivot.drop(['IND_CONFLICT', 'Country_Name', 'Time', 'Political Stability and Absence of Violence/Terrorism: Estimate'], axis=1).apply(pd.to_numeric, errors='coerce')
y = df_pivot['IND_CONFLICT']


In [0]:
# Impute missing values in the feature set by linear inter- and extrapolation
X = X.interpolate(method='linear', limit_direction='both')

In [0]:
# Confirm that the feature set does not contain NaN values anymore
display(X.isna().sum())

In [0]:
# Perform a train-test split on the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [0]:
# Initialize the random forest classifier. Set max_depth to 5 to avoid overfitting
model_rf = RandomForestClassifier(max_depth=5, random_state=42)
model_rf.fit(X_train, y_train)

In [0]:
# Predict the target based on the test data of the feature set and print the classification report
y_pred = model_rf.predict(X_test)
print(classification_report(y_test, y_pred))

The precision, recall and f1-score are relatively high for the 0-label and quite a bit lower for the 1-label, thus showing potential class imbalance and a difficulty to correctly predict a conflict in a given year and within a given country.

In [0]:
# Print the confusion matrix to see the true and false positives and negatives
confusion = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=confusion).plot()

Confusion matrix showsm that 32 true unstable situations were not picked up by the model, which is suboptimal. Also, 11 stable situations were deemed unstable by the model, which is equally suboptimal. Luckily, we still find most cases on the main diagonal, however.

In [0]:
# Calculate the evaluation metrics (precision, recall, accuracy and F1-score)
a = accuracy_score(y_test, y_pred)
r = recall_score(y_test, y_pred)
p = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy of the model", a)
print("Precision of the model", p)
print("Recall of the model", r)
print('F1-score of the model', f1)

An accuracy of 0.88 is not optimal, but also not bad, although the costs of a false negative (not predicting a conflict when there is one) and a false positive (predicting a conflict when there is none) are quite costly. One could say that avoiding false negatives is even more costly, so recall is the most important measure here. At 0.71 rounded to two decimals, this measure does not show a particularly strong model. When looking at the F1-score, which takes into account class imbalance, recall ánd precision, a score of 0.78 is intermediate.

In [0]:
# Create the SHAP explainer for the Random Forest model
explainer = shap.TreeExplainer(model_rf)

# Calculate SHAP values for the test set
shap_values = explainer.shap_values(X_test)

In [0]:
# Generate a SHAP summary plot with larger size for better visibility
shap.summary_plot(shap_values[0:, 0:, 0], X_test, plot_size=(12, 8))

The feature importance plot shows that the merchandise trade and forest area variables are the features with the highest feature importance. The more merchandise trade or forest area, the higher the probability of a conflict, which is in accordance with prior research (Muchlinski et al., 2016). Interestingly, military expediture or armed personnel is not the most valuable predictor of conflict situations!